In [1]:
from operator import itemgetter
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import itertools
from itertools import chain, combinations
import numpy as np
import re
from nltk.stem import PorterStemmer 
from collections import Counter
import math
from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import inv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import time
from operator import itemgetter
from Levenshtein import distance
from operator import itemgetter as it
from itertools import repeat
from datasketch import MinHash, MinHashLSH
import time
import ast
from collections import defaultdict 

In [2]:
numberOfProfiles= 4

In [3]:
recipesFile = "real_recipe_dataset.json"

In [4]:
syntheticDataGrocery = False

In [5]:
stopwords =["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]

In [6]:
ps = PorterStemmer()

In [7]:
WORDS = {}

In [8]:
#GIVEN A BASKET RETURN ALL ITEMS THAT THAT ARE RELATED TO THE GIVEN BASKET BY THE ASSOCIATION RULES
#THE ITEMS THAT ARE ALREADY IN THE BASKET WILL NOT BE RETURNED
def consequentsPrediction(inputs):
    #print("ORIGINAL" + str(inputs))    
    consequent_insieme = set()
    for left_rule in rules["antecedents"].values :      
        if left_rule.issubset(inputs):
            froz_set = rules[rules['antecedents'] == left_rule ]["consequents"].values
            test = set(itertools.chain.from_iterable(froz_set))
            for k in test :
                consequent_insieme.add(k)
    for elem in inputs:
        if elem in consequent_insieme:
            consequent_insieme.remove(elem)
    #print("PREDICTED"+ str(consequent_insieme))
    return consequent_insieme

In [9]:
#Clean the string x
def cleaningString (x):
    result = []
    for j in x :
        j = j.lower()
        j = re.sub(r"[^a-zA-Z]+",' ',j)
        j = j.strip()
        result.append(j)
    return result

In [10]:
def stemming(x):
    result = []
    for j in x:
        temp = j.split()
        temp2 = ""
        for i in temp:
            stem = ps.stem(i)
            temp2 += stem
            temp2 += " "
        temp2 = temp2.strip()
        result.append(temp2)
    return result

In [11]:
def edits_one(word):
    return_set = set()
    for w in WORDS:
        temp =  distance(word,w)
        if (temp<=1):
            return_set.add(w)
            
    return return_set

In [12]:
def edits_two(word):
    return_set = set()
    for w in WORDS:
        temp =  distance(word,w)
        if (temp<=2):
            return_set.add(w)
            
    return return_set

In [13]:
def words(text): return re.findall(r'\w+', text.lower())

In [14]:
def correction(word): 
    "Most probable spelling correction for word."
    result = candidates(word)
    if len(result) == 0:
        return ""
    return max(candidates(word), key=P)

In [15]:
#Clean the string x
def cleaningStringImproved (x):
    result = []
    for j in x :
        j = j.lower()
        j = re.sub(r"[^a-zA-Z]+",' ',j)
        j = j.strip()
        result.append(j)
    return " ".join(x).strip().lower()

In [16]:
# Function to convert   
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s).strip()) 

In [17]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or edits_one((word)) or edits_two((word)))

In [18]:
#Clean the string x
def deleteStopwords (x):
    result = []
    for j in x :
        temp = j.split()
        temp2 = ""
        for i in temp:
            if i in stopwords:
                i = " "
            elif len(i) <= 2 :
                i = " "
            temp2 += i
            temp2 += " "
        temp2 = temp2.strip()
        result.append(temp2)
    return result

In [19]:
def correctionString(x):
    result = []
    for j in x:
        temp = j.split()
        temp2 = ""
        for i in temp:
            correct = correction(i)
            temp2 += correct
            temp2 += " "
        temp2 = temp2.strip()
        if(temp2 != ""):
            result.append(temp2)
    return result

In [20]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [21]:
def basketQuery(b):
    global lsh
    global lsh2
    query = b.split()
    m4 = MinHash(num_perm=128)
    for d in query:
        m4.update(d.encode('utf8'))
    result = lsh.query(m4)
    if len(result) == 0 :
        result = lsh2.query(m4)
    if len(result)> 0 :
        converted = [(b,recipes_cuisines[int(s)], recipes_collection[int(s)] ,m4.jaccard(minhash[int(s)])) for s in result]
        converted.sort(key=itemgetter(3), reverse = True)
        result = converted
    return result[:10]

In [22]:
#STARTING
recipes = pd.DataFrame()
if syntheticDataGrocery!= True :
    groceries = pd.DataFrame()
    
    groceries = pd.read_csv("real_market_basket_dataset.csv")
    
    #Drop the columns "Item(s)" in groceries and "id" in recipes
    groceries = groceries.drop("Item(s)", axis = 1)
    stripped_nan = []
    
    for row in range(0, groceries.count()[0]):
        string_tmp_1 = pd.Series()
    
        # build column with single ingredients
        string_tmp_1 = groceries.iloc[row, :].dropna()
        string_tmp_2 = string_tmp_1.reset_index(drop = True)
        string_tmp_3 = list(string_tmp_2.to_numpy())
        stripped_nan.append(string_tmp_3)
    
    groceries = pd.DataFrame(pd.Series(stripped_nan).reset_index()).drop("index",axis=1).set_axis(["original basket"], 1, inplace = False)
    

In [23]:
recipes = pd.read_json(recipesFile) 
recipes = recipes.drop("id",axis=1)

In [24]:
start_time = time.time()

In [25]:
lista_prova = [cleaningString(s) for s in recipes["ingredients"]]
recipes["ingredients"] = lista_prova

In [26]:
lista_prova = [cleaningString(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [27]:
lista_prova = [deleteStopwords(s) for s in recipes["ingredients"]]
recipes["ingredients"] = lista_prova

In [28]:
lista_prova = [deleteStopwords(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [29]:
temporary = " ".join(list(itertools.chain.from_iterable(recipes["ingredients"])))

In [30]:
WORDS = Counter(words(temporary))

In [31]:
#temporary location, need to move this function in the function section
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

In [32]:
lista_prova = [correctionString(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [33]:
#stemming ingredients
lista_prova=[]
for recipe in recipes["ingredients"]:
    lista_prova.append(stemming(recipe))
recipes["ingredients"] = lista_prova

In [34]:
#stemming groceries
lista_prova=[]
for grocery in groceries["original basket"] :
    lista_prova.append(stemming(grocery))
groceries["original basket"] = lista_prova

In [35]:
stripped_nan = groceries["original basket"].values

In [36]:
te = TransactionEncoder()
te_ary = te.fit(stripped_nan).transform(stripped_nan)
one_hot_encoded = pd.DataFrame(te_ary, columns = te.columns_)

In [37]:
#THE DATA IS READY TO BE APPLIED TO A-PRIORI ALGORITHM. THE MINIMU SUPPORT IS 0.01, THAT IS 1%
results_apriori = apriori(one_hot_encoded, min_support=0.01,
                          use_colnames=True, verbose = 0)

In [38]:
#GET ALL THE ASSOCIATION RULES FROM THE RESULT OF THE A-PRIORI ALGORITHM  WITH CONFIDENCE ABOVE 0.5.
rules = association_rules(results_apriori, metric = 'confidence', 
                                      min_threshold = 0.5)

In [39]:
rules["interest"] = rules["confidence"] - rules["consequent support"]

In [40]:
rules = rules[ (rules['interest'] >= 0.35) ]

In [41]:
items = sorted(
    set(itertools.chain.from_iterable(rules["antecedents"].values)))

In [42]:
predicted_baskets = []
total_baskets = []
for i in range(len(groceries["original basket"])):
    result = list(consequentsPrediction(groceries["original basket"][i]))
    predicted_baskets.append(result)
    total_baskets.append(groceries["original basket"][i]+result)

In [43]:
groceries["association items"] = pd.Series(predicted_baskets).reset_index().drop("index",1)

In [44]:
groceries["original basket + association items"] = pd.Series(total_baskets).reset_index().drop("index",1)

In [45]:
lista_prova = [listToString(s) for s in groceries["original basket + association items"]]
groceries["document basket"] = lista_prova

In [46]:
recipes

,cuisine,ingredients
0,greek,"[romain lettuc, black oliv, grape tomato, garl..."
1,southern_us,"[plain flour, ground pepper, salt, tomato, gro..."
2,filipino,"[egg, pepper, salt, mayonais, cook oil, green ..."
3,indian,"[water, veget oil, wheat, salt]"
4,indian,"[black pepper, shallot, cornflour, cayenn pepp..."
...,...,...
39769,irish,"[light brown sugar, granul sugar, butter, warm..."
39770,italian,"[kraft zesti italian dress, purpl onion, brocc..."
39771,irish,"[egg, citru fruit, raisin, sourdough starter, ..."
39772,chinese,"[boneless chicken skinless thigh, minc garlic,..."


In [47]:
recipes_collection = []
for list_recipe in recipes["ingredients"]:
        string = ""
        string = ", ".join(list_recipe)
        string = string.strip()
        recipes_collection.append(string)

In [48]:
groceries_documents = groceries["document basket"].values.tolist()

In [49]:
lista_prova = [listToString(s) for s in recipes["ingredients"]]
recipes["document recipe"] = lista_prova

In [50]:
recipes_documents = recipes["document recipe"].values.tolist()

In [51]:
recipes_cuisines = recipes["cuisine"].values.tolist()

In [52]:
minhash = []

In [53]:
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh2 = MinHashLSH(threshold=0.2, num_perm=128)

In [54]:
minhash = []
index = 0
for text in recipes_documents:
    tokens = text.split()
    m = MinHash(num_perm=128)
    for s in tokens:
        m.update(s.encode('utf8'))
    minhash.append(m)
    lsh.insert(str(index), m)
    lsh2.insert(str(index), m)
    index += 1

In [55]:
test = [basketQuery(document) for document in groceries_documents]

In [56]:
pd.DataFrame(test).to_csv("resultMinHashLSH.csv", index = False)

In [57]:
combined_csv = pd.DataFrame(test)

In [58]:
#Input : index of the element, the most N common in the column
def customerProfiling(index, index_common):
    temp = combined_csv[index].dropna()
    out = [recipe for basket,cuisine,recipe,cosine in temp ] 
    recipes_common = Counter(out)
    recipes_looking, value_cos = recipes_common.most_common(40)[index_common]
    output = defaultdict(float) 
    for b, c, r, cos in temp: 
        output[r] += cos 
    return [(recipe, cos/value_cos) for recipe,cos in list(output.items()) if recipe == recipes_looking] 

In [59]:
totalProfiles= []
for n in range(numberOfProfiles):
    customerProfiles = []
    for i in range(4):
        result = customerProfiling(i,n)[0]
        customerProfiles.append(result)
    totalProfiles.append(customerProfiles)

In [60]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 248.77400422096252 seconds ---


In [61]:
f=open('finalReportMinHashLSH.txt','w')
index_write = 1
f.write("Customer profiling based on cosine similarity score\n")
f.write("Max number of recipes per profile: 4\n\n\n\n")
for profile in totalProfiles:
    index_recipe = 1
    f.write("Customer Profile "+ str(index_write))
    f.write("\n")
    f.write("We have customers who plan to make the following recipes :\n")
    for elem in profile:
        recipe, cosine = elem
        f.write(str(index_recipe) + ") " + recipe + " (Avg.Score: "+ str(cosine)+ ")\n")
        index_recipe+=1
    f.write("\n\n\n\n")
    index_write+=1
f.close()